## Installation and configuration¶

This notebook configures the notebooks in this tutorial to connect to an Azure Machine Learning (AzureML) Workspace. You can use an existing workspace or create a new one.

In [1]:
import azureml.core
from azureml.core import Workspace
from dotenv import set_key, get_key, find_dotenv
from pathlib import Path
from testing_utilities import get_auth
import json
import time

The AML Python SDK is already installed. Let's check the AML SDK version.

In [2]:
print("SDK Version:", azureml.core.VERSION)

SDK Version: 1.0.21


## Set up your Azure Machine Learning workspace

To create or access an Azure ML Workspace, you will need the following information:

* An Azure subscription id
* A resource group name
* A name for your workspace
* A region for your workspace

We also require you to provide variable names that will be used to create images, deployment computes, etc. in later notebooks.

**Note**: As with other Azure services, there are limits on certain resources like cluster size associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

Replace the values in the following cell with your information. If you would like to use service principal authentication as described [here](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azure-ml.ipynb) make sure you provide the optional values as well.

In [1]:
# Azure resources
subscription_id = "<subscription_id>"
resource_group = "<resource_group>"  
workspace_name = "<workspace_name>"
workspace_region = "resource_region" # e.g. resource_region = "eastus"

# IoT hub name - a globally UNIQUE name is required, e.g. iot_hub_name = "myiothubplusrandomnumber".
iot_hub_name = "<iot_hub_name>" 
device_id = "<device_id>" # the name you give to the edge device. e.g. device_id = "mygpudevice"

image_name = "<image_name>" 

Create and initialize a dotenv file for storing parameters used in multiple notebooks.

In [7]:
env_path = find_dotenv()
if env_path == "":
    Path(".env").touch()
    env_path = find_dotenv()

In [ ]:
set_key(env_path, "subscription_id", subscription_id)
set_key(env_path, "resource_group", resource_group)
set_key(env_path, "workspace_name", workspace_name)
set_key(env_path, "workspace_region", workspace_region)

set_key(env_path, "image_name", image_name)

set_key(env_path, "iot_hub_name", iot_hub_name)
set_key(env_path, "device_id", device_id)

## Create Azure Resources

In [ ]:
# login in your account
accounts = !az account list --all -o tsv
if "Please run \"az login\" to access your accounts." in accounts[0]:
    !az login -o table
else:
    print("Already logged in")

Below we will reload it just to make sure that everything is working.

In [ ]:
!az account set --subscription $subscription_id 

### Create the workspace
This cell will create an AzureML workspace for you in a subscription, provided you have the correct permissions.
This will fail when:

1. You do not have permission to create a workspace in the resource group
2. You do not have permission to create a resource group if it's non-existing.
3. You are not a subscription owner or contributor and no Azure ML workspaces have ever been created in this subscription

If workspace creation fails, please work with your IT admin to provide you with the appropriate permissions or to provision the required resources. If this cell succeeds, you're done configuring AzureML!

In [ ]:
# import the Workspace class and check the azureml SDK version
# from azureml.core import Workspace

ws = Workspace.create(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group,
    location=workspace_region,
    create_resource_group=True,
    auth=get_auth(),
    exist_ok=True,
)
# persist the subscription id, resource group name, and workspace name in aml_config/config.json.
ws.write_config()

Below we will reload it just to make sure that everything is working.

In [ ]:
# load workspace configuratio from ./aml_config/config.json file.
ws = Workspace.from_config(auth=get_auth())
ws.get_details()

### Create IoT Hub

In [ ]:
# install az-cli iot extension - I had to use "sudo -i" to make it work
!sudo -i az extension add --name azure-cli-iot-ext

In [ ]:
!az iot hub list --resource-group $resource_group -o table

In [ ]:
# Command to create a Standard tier S1 hub with name `iot_hub_name` in the resource group `resource_group`.
!az iot hub create --resource-group $resource_group --name $iot_hub_name --sku S1

In [ ]:
# Command to create a free tier F1 hub. You may encounter error "Max number of Iot Hubs exceeded for sku = Free" if quota is reached.
# !az iot hub create --resource-group $resource_group --name $iot_hub_name --sku F1

### Register an IoT Edge device
In the Azure cloud shell, enter the following command to create a device with name device_id in your iot hub.

In [ ]:
time.sleep(20) # Wait 20 seconds before creating a device
print("az iot hub device-identity create --hub-name {} --device-id {} --edge-enabled -g {}".format(iot_hub_name,device_id,resource_group))

In [ ]:
!az iot hub device-identity create --hub-name $iot_hub_name --device-id $device_id --edge-enabled -g $resource_group

Obtain device_connection_string. It will be used in the next step.

In [ ]:
print("az iot hub device-identity show-connection-string --device-id {} --hub-name {} -g {}".format(device_id, iot_hub_name,resource_group))

In [ ]:
json_data = !az iot hub device-identity show-connection-string --device-id $device_id --hub-name $iot_hub_name -g $resource_group
print(json_data)

In [ ]:
device_connection_string = json.loads(''.join([i for i in json_data if 'WARNING' not in i]))['connectionString']
print(device_connection_string)

In [ ]:
set_key(env_path, "device_connection_string", device_connection_string)

In this notebook, we created a ".env" file to save and reuse the variables needed cross all the notebooks. We also created a new Azure resource group with name <resource\_group>, where all Azure resources are created. We can now move on to the next notebook [02_IoTEdgeConfig.ipynb](02_IoTEdgeConfig.ipynb).